In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as sk_text

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os



# Set the desired TensorFlow output level for this example
# tf.logging.set_verbosity(tf.logging.ERROR)


In [2]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)



# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [3]:
"""path = "/content/drive/MyDrive/yelp_dataset/"
    
filename = os.path.join(path,"business_data.csv")    
dfb = pd.read_csv(filename, na_values=['NA', '?'])


removeLessThan20 = dfb[dfb['review_count'] < 20].index
dfb.drop(removeLessThan20, inplace=True)
dfb[0:3]"""

'path = "/content/drive/MyDrive/yelp_dataset/"\n    \nfilename = os.path.join(path,"business_data.csv")    \ndfb = pd.read_csv(filename, na_values=[\'NA\', \'?\'])\n\n\nremoveLessThan20 = dfb[dfb[\'review_count\'] < 20].index\ndfb.drop(removeLessThan20, inplace=True)\ndfb[0:3]'

In [4]:
"""path = "/content/drive/MyDrive/yelp_dataset/"

filename = os.path.join(path, "reviews_merged.csv")
df_b = pd.read_csv(filename)

df_b"""

'path = "/content/drive/MyDrive/yelp_dataset/"\n\nfilename = os.path.join(path, "reviews_merged.csv")\ndf_b = pd.read_csv(filename)\n\ndf_b'

In [5]:
"""merged_df = pd.merge(df_b, dfb, on='business_id')
merged_df[0:5]"""

"merged_df = pd.merge(df_b, dfb, on='business_id')\nmerged_df[0:5]"

In [6]:
"""merged_df.to_csv(r'/content/drive/MyDrive/yelp_dataset/merged_business_and_review.csv', index = False)"""

"merged_df.to_csv(r'/content/drive/MyDrive/yelp_dataset/merged_business_and_review.csv', index = False)"

In [7]:
"""merged_df[12345:12348]"""

'merged_df[12345:12348]'

In [8]:
"""path = "/content/drive/MyDrive/yelp_dataset/"
    
filename = os.path.join(path,"merged_business_and_review.csv")    
dfa = pd.read_csv(filename)

dfa.drop(['business_id', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'is_open', 'hours', 'attributes', 'categories'], axis=1, inplace=True)
dfa[0:3]"""

'path = "/content/drive/MyDrive/yelp_dataset/"\n    \nfilename = os.path.join(path,"merged_business_and_review.csv")    \ndfa = pd.read_csv(filename)\n\ndfa.drop([\'business_id\', \'address\', \'city\', \'state\', \'postal_code\', \'latitude\', \'longitude\', \'is_open\', \'hours\', \'attributes\', \'categories\'], axis=1, inplace=True)\ndfa[0:3]'

In [9]:
"""dfa.to_csv(r'/content/drive/MyDrive/yelp_dataset/transformed_data.csv', index = False)"""

"dfa.to_csv(r'/content/drive/MyDrive/yelp_dataset/transformed_data.csv', index = False)"

In [10]:
path = '/content/drive/MyDrive/yelp_dataset/'
filename_read = os.path.join(path, 'transformed_data.csv')
df = pd.read_csv(filename_read, na_values=['NA', '?'])
df.drop(['business_id', 'attributes', 'categories'], axis=1, inplace=True)

In [ ]:
data_lite = df[:-51918]
business_names = data_lite['name']
data_lite.drop(['name'], axis=1, inplace=True)
data_lite[0:1]

In [31]:
vectorizer = sk_text.TfidfVectorizer(max_features=2000, min_df=2, max_df=750, strip_accents='ascii')
matrix = vectorizer.fit_transform(data_lite['all_reviews'])



In [32]:
tfidf_data = matrix.toarray()
"""print(tfidf_data.shape)
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))"""

data_1 = tfidf_data
df_1 = pd.DataFrame(data=data_1, columns=vectorizer.get_feature_names_out())
df_1

,10am,11pm,120,1905,2011,2012,2021,250,30th,31,...,yelper,yikes,yoga,yolk,yours,yr,yrs,zinburger,zoo,zucchini
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.017142,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.008982,0.0,0.0,0.0,0.000000,0.004555,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9999,0.0,0.0,0.000000,0.0,0.0,0.0,0.009044,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.009207,0.0,0.0,0.0,0.0


In [34]:
data_lite2 = pd.merge(df_1, data_lite, left_index=True, right_index=True)
data_lite2.drop('all_reviews', axis=1, inplace=True)
data_lite2

,10am,11pm,120,1905,2011,2012,2021,250,30th,31,...,yoga,yolk,yours,yr,yrs,zinburger,zoo,zucchini,stars,review_count
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.5,24
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,27
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5.0,29
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.5,23
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.5,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.5,23
9997,0.0,0.0,0.017142,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.5,63
9998,0.0,0.0,0.008982,0.0,0.0,0.0,0.000000,0.004555,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.0,106
9999,0.0,0.0,0.000000,0.0,0.0,0.0,0.009044,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.009207,0.0,0.0,0.0,0.0,4.5,108


In [38]:
#tfidf_data = tfidf_data.to_numpy()
x, y = to_xy(data_lite2, "stars")

print(x.shape)
print(y.shape)

(10001, 2001)
(10001,)


In [ ]:
model = Sequential()
model.add(Dense(30, input_dim=x.shape[1], activation='relu'))
model.add(Dense(20, activation='relu'))